# Advanced Lane Finding Project

The goals / steps of this project are the following:

1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
2. Apply a distortion correction to raw images.
3. Use color transforms, gradients, etc., to create a thresholded binary image.
4. Apply a perspective transform to rectify binary image ("birds-eye view").
5. Detect lane pixels and fit to find the lane boundary.
6. Determine the curvature of the lane and vehicle position with respect to center.
7. Warp the detected lane boundaries back onto the original image.
8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


## 1. Implementation

This project was developed to dynamicaly calibrate each step and feature implemented on it through opencv trackbars. This way it is easier to understand and interact with each method implemented and directly see what happens in the output of each step.

Run the following cells and follow the instructions specified to interact with this simulation.

Complete explanation (writeup) is on the README.md 

## 1.1. Libraries

In [6]:
# Import libs used in this project
import numpy as np
import time
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

## 1.2. Global Variables

In [7]:

# Initialize global parameters with previous calibrated values

# Define globaly original image size
img_size = [1280,720]

# Color and Gradient Threshold
hls_threshold = (160,255)
sobel_threshold = (19,105)

# Warp perspective auxiliar variables to compute source and destination vertices to be used by cal_perspective_matrix()
dst_horizontal_offset = 360
src_roi_upper = 450
src_horizontal_offset_upper = 47
src_horizontal_offset_lower = 457
src_horizontal_drift_upper_l = 0
src_horizontal_drift_upper_r = 0
src_horizontal_drift_lower_l = 0
src_horizontal_drift_lower_r = 0

# Sliding Window hyperparams:
nwindows = 9
margin = 100
minpix = 50

# Search Around hyperparams:
margin_sa = 100

# moving Average max length:
moving_avg_max_count = 12

# Initialize Lanes polynomial fit average arrays with empty values
left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

# Define conversions in x and y from pixels space to meters
lane_lines_dist_base = 690
ym_per_px = 30/720 # meters per pixel in y dimension
xm_per_px = 3.7/690 # meters per pixel in x dimension


## 1.3. Functions

In [8]:
# Compute and return the camera calibration matrix and distortion coefficients using a set of chessboard images
def cal_camera_calibration(calibration_imgs_glob_path='camera_cal/calibration*.jpg', num_vertices_x=9, num_vertices_y=6,show=False):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((num_vertices_y*num_vertices_x,3), np.float32)
    objp[:,:2] = np.mgrid[0:num_vertices_x,0:num_vertices_y].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(calibration_imgs_glob_path)

    print("Start camera calibration...")

    # Step through the list and search for chessboard corners
    for fname in images:
        img_camera_calibration = cv2.imread(fname)
        gray = cv2.cvtColor(img_camera_calibration,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (num_vertices_x,num_vertices_y),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            if show:
                img_camera_calibration = cv2.drawChessboardCorners(img_camera_calibration, (num_vertices_x,num_vertices_y), corners, ret)
                cv2.imshow('Camera calibration',img_camera_calibration)
                cv2.waitKey(100)

    if show:
        cv2.destroyWindow("Camera calibration")

    # Get calibration image shape
    cal_img = cv2.imread(images[0])
    cal_img_shape = cal_img.shape[1::-1]

    # Calculate camera calibration params
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, cal_img_shape, None, None)

    print("Camera calibrated! RMS re-projection error (retval): {}".format(ret))

    return ret, mtx, dist


# Returns undistorted image using the camera intrinsic and extrinsic parameters previously calculated
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


# Return the combined binary image of Sobel and S Channel thresholded
def cal_threshold(img,hls_threshold=(170, 255),sobel_threshold=(20, 100),color_space="RGB"):

    # Convert to HLS color space and separate the V channel
    if color_space == "RGB":
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    else:
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobel_threshold[0]) & (scaled_sobel <= sobel_threshold[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= hls_threshold[0]) & (s_channel <= hls_threshold[1])] = 1

    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary,sxbinary,s_binary,h_channel,l_channel,s_channel


# Calculate and return the perspective transform matrix and source/destination vertices arrays
def cal_perspective_matrix(img_size_x,img_size_y):
    # Define warp source vertices 
    src_vertices = np.array(
    [[img_size_x/2 - src_horizontal_offset_lower + src_horizontal_drift_lower_l , img_size_y],
        [img_size_x/2 - src_horizontal_offset_upper + src_horizontal_drift_upper_l , src_roi_upper], 
        [img_size_x/2 + src_horizontal_offset_upper + src_horizontal_drift_upper_r , src_roi_upper], 
        [img_size_x/2 + src_horizontal_offset_lower + src_horizontal_drift_lower_r , img_size_y]],
        np.float32)

    # Define warp destination vertices 
    dst_vertices = np.array(
    [[img_size_x/2 - dst_horizontal_offset, img_size_y],
        [img_size_x/2 - dst_horizontal_offset, 0],
        [img_size_x/2 + dst_horizontal_offset, 0], 
        [img_size_x/2 + dst_horizontal_offset, img_size_y]],
        np.float32)

    # Calculate the warp transformation matrix M
    M = cv2.getPerspectiveTransform(src_vertices, dst_vertices)
    
    return src_vertices, dst_vertices, M

# Return the perspective transform of the given image
def cal_perspective(img_to_warp, M, inverted=False):
    
    # get input image proper width-height size accordingly to it's shape
    img_to_warp_size = None
    if len(img_to_warp.shape) > 2:
        img_to_warp_size = img_to_warp.shape[1::-1]
    else:
        img_to_warp_size = img_to_warp.shape[::-1]
        img_to_warp = img_to_warp*255
    
    if inverted:
        # Warp the image using the inverted perspective transform matrix flag
        warped = cv2.warpPerspective(img_to_warp, M, img_to_warp_size,flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    else:
        # Warp the image using perspective transform matrix
        warped = cv2.warpPerspective(img_to_warp, M, img_to_warp_size)

    return warped


def cal_sliding_window(img_warped,nwindows=9,margin=100,minpix=50,draw=True):

    # Take a histogram of the bottom half of the image
    histogram = np.sum(img_warped[img_warped.shape[0]//2:,:], axis=0)

    # Create an output image to draw on and visualize the result
    img_binary_out = np.dstack((img_warped, img_warped, img_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img_warped.shape[0] - (window+1)*window_height
        win_y_high = img_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        if draw:
            cv2.rectangle(img_binary_out,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(img_binary_out,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fit, right_fit, left_fitx, right_fitx, fity = cal_fit_polynomial(img_warped.shape[0],leftx,lefty,rightx,righty,clear_ma=True)

    if draw:
        # Color in left and right line pixels
        img_binary_out[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        img_binary_out[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Draw the left and right polynomials on top of the img_binary_out image
        if len(left_fitx) > 0 and len(right_fitx) > 0 :
            lane_l = np.dstack((left_fitx, fity))[0]
            lane_r = np.dstack((right_fitx, fity))[0]
            cv2.polylines(img_binary_out,np.int32([lane_l]),False,(0,255,255),thickness=3)
            cv2.polylines(img_binary_out,np.int32([lane_r]),False,(0,255,255),thickness=3)

    return left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out


def cal_search_around_poly(img_warped, margin_sa=100, draw=True):

    # Create an image to draw on and an image to show the selection window
    img_binary_out = np.dstack((img_warped, img_warped, img_warped))*255

    # Grab activated pixels
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Calculate the currente lane lines fit averages
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    left_fit_avg = np.array([])
    right_fit_avg = np.array([])

    if len(left_fit_avg_arr_filtered)>0 and len(right_fit_avg_arr_filtered)>0:
        left_fit_avg = np.mean(left_fit_avg_arr_filtered,axis=0)
        right_fit_avg = np.mean(right_fit_avg_arr_filtered,axis=0)
    
    # If any average arrays is empty, then return with empty values. This might be an error durting calibration
    else:
        return left_fit_avg, right_fit_avg, np.array([]), np.array([]), np.array([]), img_binary_out

    # Set the area of search based on activated x-values
    left_lane_inds = ((nonzerox > (left_fit_avg[0]*(nonzeroy**2) + left_fit_avg[1]*nonzeroy + 
                    left_fit_avg[2] - margin_sa)) & (nonzerox < (left_fit_avg[0]*(nonzeroy**2) + 
                    left_fit_avg[1]*nonzeroy + left_fit_avg[2] + margin_sa)))
    right_lane_inds = ((nonzerox > (right_fit_avg[0]*(nonzeroy**2) + right_fit_avg[1]*nonzeroy + 
                    right_fit_avg[2] - margin_sa)) & (nonzerox < (right_fit_avg[0]*(nonzeroy**2) + 
                    right_fit_avg[1]*nonzeroy + right_fit_avg[2] + margin_sa)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Calculate the new fit polynomial
    left_fit, right_fit, left_fitx, right_fitx, fity = cal_fit_polynomial(img_warped.shape[0],leftx,lefty,rightx,righty)

    if draw:
        window_img = np.zeros_like(img_binary_out)

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin_sa, fity]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin_sa, 
                                fity])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin_sa, fity]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin_sa, 
                                fity])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        img_binary_out = cv2.addWeighted(img_binary_out, 1, window_img, 0.3, 0)

        # Color in left and right line pixels
        img_binary_out[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        img_binary_out[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Draw the left and right polynomials on top of the img_binary_out image
        if len(left_fitx) > 0 and len(right_fitx) > 0 :
            lane_l = np.dstack((left_fitx, fity))[0]
            lane_r = np.dstack((right_fitx, fity))[0]
            cv2.polylines(img_binary_out,np.int32([lane_l]),False,(0,255,255),thickness=3)
            cv2.polylines(img_binary_out,np.int32([lane_r]),False,(0,255,255),thickness=3)
    
    
    return left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out


def cal_fit_polynomial(img_size_y, leftx, lefty, rightx, righty, clear_ma=False):
    global left_fit_avg_arr
    global right_fit_avg_arr

    # Reset lanes polynomial fit average arrays if requested
    if clear_ma:
        left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
        right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

    # Calculate new fit polynomial values
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
    except TypeError:
        left_fit = []
    
    try:
        right_fit = np.polyfit(righty, rightx, 2)
    except TypeError:
        right_fit = []

    # Append the new value to the moving average array
    left_fit_avg_arr.append(left_fit) 
    right_fit_avg_arr.append(right_fit) 

    # Remove first (older) value and trim array to max moving average elements
    left_fit_avg_arr = left_fit_avg_arr[1:moving_avg_max_count+1]
    right_fit_avg_arr = right_fit_avg_arr[1:moving_avg_max_count+1]
    
    # Filter only not empty arrays
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    # Build 2 element arrays to store left and right lane line variables together
    lanes_fit = [left_fit_avg_arr_filtered,right_fit_avg_arr_filtered]
    lanes_fit_avg = [np.array([]),np.array([])]
    lanes_fitx = [np.array([]),np.array([])]

    # Generate x and y values for plotting
    fity = np.linspace(0, img_size_y-1, img_size_y)

    # Calculate average fit and x values for each lane side
    for i in range(len(lanes_fit)):
        if len(lanes_fit[i]) > 0:
            try:
                lanes_fit_avg[i] = np.mean(lanes_fit[i],axis=0)
                lanes_fitx[i] = lanes_fit_avg[i][0]*fity**2 + lanes_fit_avg[i][1]*fity + lanes_fit_avg[i][2]
            except TypeError:
                print('cal_fit_polynomial(): Failed to fit {} line!'.format('left' if i == 0 else 'right'))
                lanes_fitx[i] = np.array([])

    return lanes_fit_avg[0], lanes_fit_avg[1], lanes_fitx[0], lanes_fitx[1], fity


# Draw the lane lines area on the undistorted image
def draw_lane_area(img_undistorted, M, img_warped, left_fit, right_fit, left_fitx, right_fitx, fity):
    if len(img_warped.shape) <= 2:
        img_warped_filtered = np.dstack((img_warped, img_warped, img_warped))
    else:
        img_warped_filtered = np.copy(img_warped)

    white_threshold = (img_warped_filtered[:,:,0] > 0 ) & (img_warped_filtered[:,:,1] > 0 ) & (img_warped_filtered[:,:,2] > 0 )
    green_threshold = (img_warped_filtered[:,:,0] == 0 ) & (img_warped_filtered[:,:,1] > 0 ) & (img_warped_filtered[:,:,2] == 0 )

    # Clear green (margin area to perform sliding window or search around) and white pixels (not considered in calculations)
    img_warped_filtered[white_threshold | green_threshold] = [0,0,0]

    window_img = np.zeros_like(img_warped_filtered)
    lane_window1 = np.array([np.transpose(np.vstack([left_fitx, fity]))])
    lane_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx, fity])))])

    lane_pts = np.hstack((lane_window1,lane_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([lane_pts]), (0,255, 0))

    # Merge Lane area with lane lines 
    img_warped_filtered = cv2.addWeighted(img_warped_filtered, 1, window_img, 0.3, 0)
    
    # Inverse warp the image using inverted perspective transform matrix
    warped_inv = cal_perspective(img_warped_filtered, M, inverted=True)

    # Find colored pixels locations
    colored_threshold = (warped_inv[:,:,0] > 0 ) | (warped_inv[:,:,1] > 0 ) | (warped_inv[:,:,2] > 0 )

    # Create a pre mask with lane area drawn on it to get stronger colors
    img_lanes_aux = cv2.addWeighted(img_undistorted, 0.6, warped_inv, 1.0, 0)
    
    # Substitute the pre mask lane area pixels in the original undistorted image, to preserve pixel values from other areas
    img_lanes = np.copy(img_undistorted)
    img_lanes[colored_threshold] = img_lanes_aux[colored_threshold] 

    return img_lanes

# Calculate lane lines curvature based on the calculated polynomial fit and xy conversion factor to meters
def cal_lane_curvature(fity, left_fit_cr, right_fit_cr, factor_x=xm_per_px, factor_y=ym_per_px):
    
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(fity)

    # Calculate new polynomial fit with converted values by factors
    # left_fit = np.array([left_fit[0]*factor_x/(factor_y**2), left_fit[1]*factor_x/factor_y, left_fit[2]*factor_x])
    # right_fit = np.array([right_fit[0]*factor_x/(factor_y**2), right_fit[1]*factor_x/factor_y, right_fit[2]*factor_x])
    
    # Build an array of x values with given polynomial fit coefficients
    left_fitx = left_fit_cr[0]*fity**2 + left_fit_cr[1]*fity + left_fit_cr[2]
    right_fitx = right_fit_cr[0]*fity**2 + right_fit_cr[1]*fity + right_fit_cr[2]

    # Multply x and y arrays for the respective factors, and then calculate the new polynomial fit coefficients
    left_fit_cr_m = np.polyfit(fity*factor_y, left_fitx*factor_x, 2)
    right_fit_cr_m = np.polyfit(fity*factor_y, right_fitx*factor_x, 2)

    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr_m[0]*y_eval*factor_y + left_fit_cr_m[1])**2)**1.5) / np.absolute(2*left_fit_cr_m[0])
    right_curverad = ((1 + (2*right_fit_cr_m[0]*y_eval*factor_y + right_fit_cr_m[1])**2)**1.5) / np.absolute(2*right_fit_cr_m[0])
    
    return left_curverad, right_curverad

# Reset lanes polynomial fit average arrays
def reset_ma():
    global left_fit_avg_arr
    global right_fit_avg_arr
    left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
    right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

# Return both lane lines sides array without empty values
def get_fit_avg_arr_filtered():
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    return left_fit_avg_arr_filtered, right_fit_avg_arr_filtered

# Compose an image array in a single image
def compose_image_arr(img_list,max_columns,title_list=[],resize_factor = 0.4):
    
    # Check if is enough images to complete the last img composition line and add blank image if needed
    if len(img_list) % max_columns > 0:
        blk = np.copy(img)*0
        for i in range(max_columns - (len(img_list) % max_columns)):
            img_list.append(blk)
            title_list.append("")

    img_list_2d=[]

    for i in range(0,len(img_list)):
        # Check if its not a colored image and stack it like a 3 channel color image
        if len(img_list[i].shape) == 2:
            img_list[i] = np.dstack((img_list[i], img_list[i], img_list[i]))
            
            # if its a binary, then scale to 255
            if np.max(img_list[i]) == 1:
                img_list[i] = img_list[i]*255
        
        # Add image name on the top left corner
        if len(title_list)>0:
            cv2.putText(img_list[i],title_list[i],(10,40),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3,cv2.LINE_AA)

        # if it is the first image of a line, add an empty list to be populated next with following images
        if (i % max_columns) == 0:
            img_list_2d.append([])
        img_list_2d[int(i/max_columns)].append(img_list[i])

    # Concatenate images making a composition of fixed number of images in width
    composed_img = cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in img_list_2d])
    
    # Resize whole composition
    composed_img_resized = cv2.resize(composed_img, (int(composed_img.shape[1]*resize_factor),int(composed_img.shape[0]*resize_factor)), interpolation = cv2.INTER_AREA)
    
    return composed_img_resized

## 1.4. Advance Lane Finding implementation
This is the full project implementation that find lane lines on videos with Trackbars on the visualization window for dynamic parameters calibration

On the visualization window, press the following letters to trigger commands as described below

- `q` exits simulation.
- `w` switch to the previous image/video.
- `e` switch to the next image/video.
- `r` reset move average array.
- `a` toggle between image/video input
- `s` saves a snapshot to folder `output_images`.
- `f` toggle between show only the result and an array of image of each step.
- `v` restart current video and start saving the output result to a video file on `/output_videos` folder. Only works on video mode (`a`)



In [9]:
# Make a list of test images
test_images = glob.glob('test_images/*.jpg')
test_images_index = 0

# Make a list of test videos
test_videos = sorted(glob.glob('*.mp4'))
test_videos_index = len(test_videos)-1
print("List of videos: {}".format(test_videos))

# Initialize cap with first video
cap = cv2.VideoCapture(test_videos[test_videos_index])

# Create an video frame index
cap_counter = 0

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_out = None
img_name = None

# One time execution functions
ret, mtx, dist = cal_camera_calibration()
src_vertices, dst_vertices, M = cal_perspective_matrix(img_size[0],img_size[1])

# image visualization flag
only_result_img_show = True
input_video_img_flag = True

# Create a window with trackbars and callback functions bolow to handle when values are changed
cv2_window_name = "Advanced Lane Finding"
cv2.namedWindow(cv2_window_name)

# Threshold trackbars
def on_hls_th_l(val):
    global hls_threshold
    hls_threshold = (val,hls_threshold[1])

def on_hls_th_u(val):
    global hls_threshold
    hls_threshold = (hls_threshold[0],val)

def on_sobel_th_l(val):
    global sobel_threshold
    sobel_threshold = (val,sobel_threshold[1])

def on_sobel_th_u(val):
    global sobel_threshold
    sobel_threshold = (sobel_threshold[0],val)

cv2.createTrackbar("S Channel Threshold Lower",cv2_window_name,hls_threshold[0],255,on_hls_th_l)
cv2.createTrackbar("S Channel Threshold Upper",cv2_window_name,hls_threshold[1],255,on_hls_th_u)
cv2.createTrackbar("Sobel Threshold Lower",cv2_window_name,sobel_threshold[0],255,on_sobel_th_l)
cv2.createTrackbar("Sobel Threshold Upper",cv2_window_name,sobel_threshold[1],255,on_sobel_th_u)

# Warp perspective Trackbar
def on_warp_dst_x_offset(val):
    global dst_vertices
    global src_vertices
    global M
    global dst_horizontal_offset
    dst_horizontal_offset = val

    # Update perspective transform variables
    src_vertices, dst_vertices, M = cal_perspective_matrix(img_size[0],img_size[1])
    
cv2.createTrackbar("Dst Horizontal Offset",cv2_window_name,dst_horizontal_offset,600,on_warp_dst_x_offset)

# Polinomial Fit Trackbars
def on_nwindows(val):
    global nwindows
    nwindows = val

def on_margin(val):
    global margin
    margin = val

def on_minpix(val):
    global minpix
    minpix = val

def on_margin_sa(val):
    global margin_sa
    margin_sa = val

def on_moving_avg_max_count(val):
    global moving_avg_max_count
    global leftt_fit_avg_arr
    global right_fit_avg_arr

    moving_avg_max_count = val

    # Check if both lanes polynomial Fit average arrays have length lower than the new one specified, then add empty arrays to fill it to desired length. If it is longer, cal_fit_polynomial() will automatically trim to new length when calculates the new average
    if len(left_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(left_fit_avg_arr)):
            left_fit_avg_arr.append([])
    if len(right_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(right_fit_avg_arr)):
            right_fit_avg_arr.append([])

cv2.createTrackbar("nwindows",cv2_window_name,nwindows,50,on_nwindows)
cv2.createTrackbar("margin",cv2_window_name,margin,200,on_margin)
cv2.createTrackbar("minpix",cv2_window_name,minpix,200,on_minpix)
cv2.createTrackbar("margin_sa",cv2_window_name,margin_sa,640,on_margin_sa)
cv2.createTrackbar("moving_avg_len",cv2_window_name,moving_avg_max_count,60,on_moving_avg_max_count)


# Main simulation loop
while True:

    # Read a frame from video of current index
    if input_video_img_flag:
        ret,img = cap.read()
        cap_counter += 1
    else:
        # Read image of current index
        img = cv2.imread(test_images[test_images_index])

    if ret == False and input_video_img_flag:
        # Release video writer
        if video_out != None:
            video_out.release()
            print("Released VideoWriter!")
            video_out = None
        
        # Restart VideoCapture with same video
        cap = cv2.VideoCapture(test_videos[test_videos_index])
        cap_counter = 0
        continue

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    # Apply color and gradient threshold to undistorted image
    img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")[0]

    # Apply perspective transform to the undistorted thresholded image
    img_warped = cal_perspective(img_und_thresholded, M)

    left_fit_avg_arr_filtered, right_fit_avg_arr_filtered = get_fit_avg_arr_filtered()

    # If both moving average arrays aren't empty
    if len(left_fit_avg_arr_filtered) > 0 and len(right_fit_avg_arr_filtered) > 0 :
        # Calculate the Search Around method to find lane lines 
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_search_around_poly(img_warped, margin_sa=margin_sa)
    else:
        # Calculate the Sliding window method to find lane lines 
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

    # Check if new polynomial fits average are empty/valid
    if len(left_fit) > 0 and len(right_fit) > 0 :
        
        # Use the nearst pixel of each line from car to calculate lane_lines_dist
        lane_lines_dist = right_fitx[-1] - left_fitx[-1]
        
        # Check if the distance between lanes not is valid (+-20% of lane_lines_dist_base)
        if lane_lines_dist < lane_lines_dist_base*0.8 or lane_lines_dist > lane_lines_dist_base*1.2:
            print("Invalid Lane lines found, reseting moving average")
            # Reset moving average to force finding new lane lines by Sliding window method
            reset_ma()

        # Draw lane area into the original undistorted image
        img_lanes = draw_lane_area(img_undistorted, M, img_warped_poly_fit, left_fit, right_fit, left_fitx, right_fitx, fity)
    
        # Calculate lane lines curvature
        left_curverad, right_curverad = cal_lane_curvature(fity,left_fit,right_fit,factor_x=xm_per_px, factor_y=ym_per_px)

        # Calculate the distance from car center to lane center and build a readable string with orientation Left/Right/Center(+-0.1m)
        lane_center_diff_x = (float(right_fitx[-1] + left_fitx[-1])/2 - float(img_warped_poly_fit.shape[1])/2)*xm_per_px
        lane_center_diff_x_str = "{:0.2f}m {}".format(np.absolute(lane_center_diff_x),"Left" if lane_center_diff_x > 0.1 else ("Right" if lane_center_diff_x < 0.1 else "Center"))

    else:
        lane_center_diff_x_str = "-"
        left_curverad = 0
        right_curverad = 0
        img_lanes = img_undistorted

    # Write calculated lane information to result image
    cv2.putText(img_lanes,"Distance from lane center: {}".format(lane_center_diff_x_str),(10,120),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(img_lanes,"Left Radius: {:0.2f}m | Right Radius: {:0.2f}m".format(left_curverad, right_curverad),(10,160),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,0,255),2,cv2.LINE_AA)

    # Get current image/video name
    if input_video_img_flag:
        img_name = test_videos[test_videos_index].split('.')[0]
    else:
        img_name = test_images[test_images_index].split('/')[1].split('.')[0]

    # Check if its to show all steps or only the result. Build the images titles accordingly
    if only_result_img_show:

        # Stack all images on a single array
        img_list=[]
        img_list.append(img)
        img_list.append(img_undistorted)
        img_list.append(img_und_thresholded)
        img_list.append(img_warped)
        img_list.append(img_warped_poly_fit)
        img_list.append(img_lanes)

        title_list = ["[{}]Original".format(img_name,cap_counter),"Undistorted","Combined Threshold","Warped","Polynomial Fit","Lane Lines"]
        img_compostion = compose_image_arr(img_list,3,title_list=title_list,resize_factor=0.40)
    else:
        title_list = ["[{}]Lane Lines".format(img_name,cap_counter)]
        img_compostion= compose_image_arr([img_lanes],1,title_list=title_list,resize_factor=0.80)

    if video_out != None:
        video_out.write(img_compostion)

    cv2.imshow(cv2_window_name,img_compostion)
    key = cv2.waitKey(1) & 0xFF

    # Quit the simulation
    if key == ord('q'):
        break

    # Reset moving average 
    elif key == ord('r'):
        reset_ma()
    
    # Open the next video
    elif key == ord('e'):
        if input_video_img_flag:
            test_videos_index += 1
            if(test_videos_index >= len(test_videos)):
                test_videos_index = 0
            cap = cv2.VideoCapture(test_videos[test_videos_index])
            cap_counter = 0
        else:
            test_images_index += 1
            if(test_images_index >= len(test_images)):
                test_images_index = 0
        reset_ma()
    
    # Open the previous video
    elif key == ord('w'):
        if input_video_img_flag:
            test_videos_index -= 1
            if(test_videos_index < 0):
                test_videos_index = len(test_videos)-1
            cap = cv2.VideoCapture(test_videos[test_videos_index])
            cap_counter = 0
        else:
            test_images_index -= 1
            if(test_images_index < 0):
                test_images_index = len(test_images)-1
        reset_ma()
    
    # Enable/Disable only show the final result image
    elif key == ord('f'):
        only_result_img_show = False if only_result_img_show else True
    
    elif key == ord('a'):
        input_video_img_flag = False if input_video_img_flag else True

        if input_video_img_flag:
            cap = cv2.VideoCapture(test_videos[test_videos_index])
            cap_counter = 0
        elif video_out != None:
            # Release video writer
            video_out.release()
            print("Released VideoWriter!")
            video_out = None
        reset_ma()

    # Save an screenshot
    elif key == ord('s'):
        ts = int(time.time())
        if input_video_img_flag:
            img_name = test_videos[test_videos_index].split('.')[0]
        else:
            img_name = test_images[test_images_index].split('/')[1].split('.')[0]
        for i in range(1,len(img_list)):
            cv2.imwrite("output_images/ts{}_{}_{}.png".format(ts,img_name,title_list[i]),img_list[i])
        cv2.imwrite("output_images/ts{}_{}_composition.png".format(ts,img_name),img_compostion)


    # Configure video_out, restart current video and start saving output to video
    elif key == ord('v'):
        if video_out == None:
            ts = int(time.time())
            reset_ma()
            if input_video_img_flag:
                img_name = test_videos[test_videos_index].split('.')[0]
                cap = cv2.VideoCapture(test_videos[test_videos_index])
                cap_counter = 0
            else:
                img_name = test_images[test_images_index].split('/')[1].split('.')[0]
            print("Start VideoWriter!")
            video_out = cv2.VideoWriter("ts{}_{}_output.avi".format(ts,img_name), fourcc, 25.0, (img_compostion.shape[1], img_compostion.shape[0]))
        else:
            video_out.release()
            print("Released VideoWriter!")
            video_out = None

cap.release()
if video_out != None:
    video_out.release()
    print("Released VideoWriter!")

cv2.destroyAllWindows()

# Print all global variables to manualy update them
print("\n\nGlobal variables calibration Results:")

print("\n# Color and Gradient Threshold:")
print("  hls_threshold: ({},{})".format(hls_threshold[0],hls_threshold[1]))
print("  sobel_threshold: ({},{})".format(sobel_threshold[0],sobel_threshold[1]))

print("\n# Warp perspective:")
print("  dst_horizontal_offset = {}".format(dst_horizontal_offset))

print("\n# Sliding Window hyperparams:")
print("  nwindows = {}".format(nwindows))
print("  margin = {}".format(margin))
print("  minpix = {}".format(minpix))

print("\n# Search Around hyperparams:")
print("  margin_sa = {}".format(margin_sa))

print("\n# moving Average max length:")
print("  moving_avg_max_count = {}".format(moving_avg_max_count))

List of videos: ['challenge_video.mp4', 'harder_challenge_video.mp4', 'project_video.mp4']
Start camera calibration...
Camera calibrated! RMS re-projection error (retval): 1.0298153371058965
Start VideoWriter!
Released VideoWriter!


Global variables calibration Results:

# Color and Gradient Threshold:
  hls_threshold: (160,255)
  sobel_threshold: (19,105)

# Warp perspective:
  dst_horizontal_offset = 360

# Sliding Window hyperparams:
  nwindows = 9
  margin = 100
  minpix = 50

# Search Around hyperparams:
  margin_sa = 100

# moving Average max length:
  moving_avg_max_count = 12


# 2. Individual Calibrations

## 2.1. Camera calibration with visualization (require `cv2.imshow`)

In [10]:
# Calculate camera calibration (Set show=False if cv2.imshow is not available)
ret, mtx, dist = cal_camera_calibration(show=True)

# Read an chessboard image
# img = mpimg.imread("camera_cal/calibration1.jpg")
img = mpimg.imread("test_images/straight_lines1.jpg")

# Undistort image
undistorted = cal_undistort(img, mtx, dist)

# Plot original image and the undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.0)

# Save output
# plt.savefig("output_images/2_1_camera_calibration_undistorted.png")
plt.savefig("output_images/2_1_test_image_undistorted.png")

print("\nret: {}".format(ret))
print("mtx:\n"+str(mtx))
print("dist:\n"+str(dist))

Start camera calibration...
Camera calibrated! RMS re-projection error (retval): 1.0298153371058965

ret: 1.0298153371058965
mtx:
[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128938e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
[[-0.24688775 -0.02373132 -0.00109842  0.00035108 -0.00258571]]


## 2.2. Color and Gradient threshold dynamic calibration

In [9]:
# Make a list of test images
test_images = glob.glob('test_images/*.jpg')
test_images_index = 0

# Create a window with trackbars and callback functions bolow to handle when values are changed
cv2.namedWindow("Threshold Calibration")

def on_hls_th_l(val):
    global hls_threshold
    hls_threshold = (val,hls_threshold[1])

def on_hls_th_u(val):
    global hls_threshold
    hls_threshold = (hls_threshold[0],val)

def on_sobel_th_l(val):
    global sobel_threshold
    sobel_threshold = (val,sobel_threshold[1])

def on_sobel_th_u(val):
    global sobel_threshold
    sobel_threshold = (sobel_threshold[0],val)

cv2.createTrackbar("Color Threshold Lower","Threshold Calibration",hls_threshold[0],255,on_hls_th_l)
cv2.createTrackbar("Color Threshold Upper","Threshold Calibration",hls_threshold[1],255,on_hls_th_u)
cv2.createTrackbar("Sobel Threshold Lower","Threshold Calibration",sobel_threshold[0],255,on_sobel_th_l)
cv2.createTrackbar("Sobel Threshold Upper","Threshold Calibration",sobel_threshold[1],255,on_sobel_th_u)

# Calculate camera calibration
ret, mtx, dist = cal_camera_calibration()

while True:
    img_list=[]

    # Read image of current index
    img = cv2.imread(test_images[test_images_index])

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    img_und_thresholded,sxbinary,s_binary,h_channel,l_channel,s_channel = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")

    img_list.append(img)
    img_list.append(img_undistorted)
    img_list.append(np.zeros_like(img))
    img_list.append(h_channel)
    img_list.append(l_channel)
    img_list.append(s_channel)
    img_list.append(sxbinary)
    img_list.append(s_binary)
    img_list.append(img_und_thresholded)

    title_list = ["[{}]Original".format(test_images[test_images_index].split('/')[1]),"Undistorted","","H Channel","L Channel","S Channel","Sobel Threshold","S Channel Threshold","Combined Threshold"]
    img_compostion = compose_image_arr(img_list,3,title_list=title_list,resize_factor=0.40)

    cv2.imshow("Threshold Calibration",img_compostion)
    key = cv2.waitKey(200) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('w'):
        test_images_index += 1
        if(test_images_index >= len(test_images)):
            test_images_index = 0
    elif key == ord('e'):
        test_images_index -= 1
        if(test_images_index < 0):
            test_images_index = len(test_images)-1
    elif key == ord('s'):
        cv2.imwrite("output_images/2_2_thresholds_calibration.png",img_compostion)

cv2.destroyAllWindows()

print("Calibrated threshold params:")
print("  hls_threshold: ({},{})".format(hls_threshold[0],hls_threshold[1]))
print("  sobel_threshold: ({},{})".format(sobel_threshold[0],sobel_threshold[1]))

Calibrated threshold params:
  hls_threshold: (160,255)
  sobel_threshold: (19,105)


## 2.3. Comparisson between original and undistorted image

In [7]:
# Calculate camera calibration
ret, mtx, dist = cal_camera_calibration()

# Read an chessboard image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

# Apply color and gradient threshold
img_thresholded = cal_threshold(img)[0]
img_und_thresholded = cal_threshold(img_undistorted)[0]

# Plot original image, the undistorted image and the respectives thresholded images
f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(12, 11))
# f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_thresholded, cmap='gray')
ax3.set_title('Thresholded Image', fontsize=15)
ax4.imshow(img_und_thresholded, cmap='gray')
ax4.set_title('Thresholded Undistorted Image', fontsize=15)

# Check differences between the original and undistorted binary thresholds
diff_binary = np.zeros_like(img_thresholded)
diff_binary[((img_thresholded == 0) & (img_und_thresholded == 1)) | ((img_thresholded == 1) & (img_und_thresholded == 0))] = 1
# diff_binary[(img_thresholded == 0) & (img_und_thresholded == 1)] = 1
color_diff_binary = np.dstack(( img_thresholded,img_und_thresholded,np.zeros_like(img_und_thresholded))) * 255

ax5.imshow(diff_binary, cmap='gray')
ax5.set_title('Binary diff Image', fontsize=15)
ax6.imshow(color_diff_binary)
ax6.set_title('Colored diff Image', fontsize=15)

plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# plt.savefig("output_images/2_3_distortion_comparisson.png")

## 2.4. Warp perspective calibration with given straight lane lines images

In [9]:
# Calculate camera calibration
ret, mtx, dist = cal_camera_calibration()

# Read a straight lane image
img = mpimg.imread("test_images/straight_lines1.jpg")
img2 = mpimg.imread("test_images/straight_lines2.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)
img_undistorted2 = cal_undistort(img2, mtx, dist)

# Warp perspective auxiliar variables to compute source and destination vertices to be used by cal_perspective_matrix()
dst_horizontal_offset = 360
src_roi_upper = 450
src_horizontal_offset_upper = 47
src_horizontal_offset_lower = 457
src_horizontal_drift_upper_l = 0
src_horizontal_drift_upper_r = 0
src_horizontal_drift_lower_l = 0
src_horizontal_drift_lower_r = 0

# Calculate source and destination vertices
src_vertices, dst_vertices, M = cal_perspective_matrix(img.shape[1],img.shape[0])

# Apply perspective transform to the undistorted image
img_warped = cal_perspective(img_undistorted,M)
img_warped2 = cal_perspective(img_undistorted2,M)


for i in range(0,len(src_vertices)):
    pos1 = (int(src_vertices[i][0]),int(src_vertices[i][1]))
    pos2 = (int(src_vertices[i-1][0]),int(src_vertices[i-1][1]))
    cv2.line(img_undistorted,pos1,pos2,(255,150,150),4)
    cv2.line(img_undistorted2,pos1,pos2,(255,150,150),4)
    cv2.circle(img_undistorted,pos1,5,(255,0,0),-1)
    cv2.circle(img_undistorted2,pos1,5,(255,0,0),-1)

for dv in dst_vertices:
    pos = (int(dv[0]),int(dv[1]))
    cv2.circle(img_undistorted,pos,10,(0,255,0),-1)
    cv2.circle(img_undistorted2,pos,10,(0,255,0),-1)

cv2.line(img_warped,(310,0),(310,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped,(1010,0),(1010,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped2,(310,0),(310,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped2,(1010,0),(1010,img_warped.shape[0]),(255,0,0),8)

# Plot original image and the undistorted image
f, ((ax1, ax2,ax3),(ax4, ax5,ax6)) = plt.subplots(2, 3, figsize=(24, 9))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_warped)
ax3.set_title('Warped Image', fontsize=15)
ax4.imshow(img2)
ax4.set_title('Original Image', fontsize=15)
ax5.imshow(img_undistorted2)
ax5.set_title('Undistorted Image', fontsize=15)
ax6.imshow(img_warped2)
ax6.set_title('Warped Image', fontsize=15)
plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.0)

# plt.savefig("output_images/2_4_perspective_transform_calibration.png")

print("\nsrc_vertices:\n")
print(src_vertices)
print("\ndst_vertices:\n")
print(dst_vertices)

Start camera calibration...
Camera calibrated! RMS re-projection error (retval): 1.0298153371058965

src_vertices:

[[ 183.  720.]
 [ 593.  450.]
 [ 687.  450.]
 [1097.  720.]]

dst_vertices:

[[ 280.  720.]
 [ 280.    0.]
 [1000.    0.]
 [1000.  720.]]


## 2.5. Polynomial fit methods
- Calculate the histogram of all columns in the lower half of the warped image to get the left and right side peak,
- Calculate the lane lines by Sliding Window method
- Calculate the lane lines by Search Around method

In [13]:
# Calculate camera calibration
ret, mtx, dist = cal_camera_calibration()

# Read a straight lane image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

# Apply color and gradient threshold
img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="RGB")[0]

# Calculate source and destination vertices
src_vertices, dst_vertices, M = cal_perspective_matrix(img.shape[1],img.shape[0])

# Apply perspective transform to the undistorted image
img_warped = cal_perspective(img_und_thresholded,M)

# Create histogram of image binary activations
histogram = np.sum(img_warped[img_warped.shape[0]//2:,:], axis=0)

# Find the peak of the left and right halves of the histogram
midpoint = np.int(histogram.shape[0]//2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

print("\nleftx_base: {}".format(leftx_base))
print("rightx_base: {}".format(rightx_base))

# Plot warped image and the calculated histogram
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
ax1.imshow(img_warped, cmap='gray')
ax1.set_title('Warped Image', fontsize=15,)
ax2.plot(histogram)
ax2.set_title('Lower Half Histogram', fontsize=15)
plt.subplots_adjust(left=0.05, right=0.95, top=0.90, bottom=0.1)
plt.show()

plt.savefig("output_images/2_5_warped_histogram.png")

# Wait 2 seconds and clear plot area
plt.pause(2.0)
plt.clf()

# Reset moving averages
reset_ma()

# Calculate the Sliding window method to find lane lines 
left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

plt.imshow(img_warped_poly_fit)
plt.title("Sliding Window Method", fontsize=15)
plt.show()

plt.savefig("output_images/2_5_sliding_window.png")

# Wait 2 seconds and clear plot area
plt.pause(2.0)
plt.clf()

# Calculate the Search Around method to find lane lines 
left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_search_around_poly(img_warped, margin_sa=margin_sa)

plt.title("Search Around Method", fontsize=15)
plt.imshow(img_warped_poly_fit)
plt.show()

plt.savefig("output_images/2_5_search_around.png")

Start camera calibration...
Camera calibrated! RMS re-projection error (retval): 1.0298153371058965

leftx_base: 329
rightx_base: 1087
